In [1]:
# ==========================================================================
# CELL 1: INSTALL LIBRARIES AND MOUNT GOOGLE DRIVE
# ==========================================================================
"""
This cell installs all required libraries for:
- YOLOv8 training and inference
- COCO dataset handling
- Data preprocessing
- GPU acceleration
"""
# Install ultralytics (YOLOv8)
!pip install -q ultralytics
# Install additional required libraries
!pip install -q opencv-python pillow numpy pandas matplotlib seaborn
!pip install -q torch torchvision torchaudio  # PyTorch (should be pre-installed on Colab, but ensured here)
!pip install -q tqdm requests  # For downloading and progress tracking
!pip install -q scikit-learn  # For metrics calculation

# Mount Google Drive to save dataset and models
from google.colab import drive
drive.mount('/content/drive')

print("✓ All libraries installed successfully!")
print("✓ Google Drive mounted successfully!")

# Verify GPU availability
import torch
print(f"\n✓ GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ All libraries installed successfully!
✓ Google Drive mounted successfully!

✓ GPU Available: True
  GPU Name: Tesla T4
  GPU Memory: 15.83 GB


In [2]:
# ==========================================================================
# CELL 2: CREATE DIRECTORY STRUCTURE IN GOOGLE DRIVE
# ==========================================================================
"""
Create organized folder structure in Google Drive to store:
1. Raw NWPU VHR-10 dataset
2. Processed dataset in YOLO format
3. Trained models and checkpoints
4. Training logs and results
"""

import os
from pathlib import Path

# Define base directories in Google Drive
base_drive = '/content/drive/My Drive'

# Create main project directory
project_dir = Path(base_drive) / 'YOLOv8_NWPU_VHR10'
project_dir.mkdir(exist_ok=True)

# Create subdirectories
dataset_raw_dir = project_dir / 'NWPU_VHR-10'  # Store raw downloaded dataset
dataset_processed_dir = project_dir / 'NWPU_VHR-10_YOLO_Format'  # Store processed YOLO dataset
models_dir = project_dir / 'Models'  # Store trained models
results_dir = project_dir / 'Results'  # Store metrics and results
runs_dir = project_dir / 'Training_Runs'  # Store training logs

# Create all directories
for directory in [dataset_raw_dir, dataset_processed_dir, models_dir, results_dir, runs_dir]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✓ Created: {directory}")

# Display directory structure
print("\n" + "="*80)
print("DIRECTORY STRUCTURE CREATED:")
print("="*80)
print(f"""
{project_dir}/
├── NWPU_VHR-10/                    (Raw dataset - will be downloaded here)
├── NWPU_VHR-10_YOLO_Format/        (Processed dataset in YOLO format)
├── Models/                         (Trained model weights)
├── Results/                        (Metrics and analysis results)
└── Training_Runs/                  (Training logs and checkpoints)
""")

# Define paths as variables for easy reference
print(f"\nKey Paths:")
print(f"  Raw Dataset: {dataset_raw_dir}")
print(f"  YOLO Dataset: {dataset_processed_dir}")
print(f"  Models: {models_dir}")
print(f"  Results: {results_dir}")


✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10
✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format
✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Models
✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Results
✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Training_Runs

DIRECTORY STRUCTURE CREATED:

/content/drive/My Drive/YOLOv8_NWPU_VHR10/
├── NWPU_VHR-10/                    (Raw dataset - will be downloaded here)
├── NWPU_VHR-10_YOLO_Format/        (Processed dataset in YOLO format)
├── Models/                         (Trained model weights)
├── Results/                        (Metrics and analysis results)
└── Training_Runs/                  (Training logs and checkpoints)


Key Paths:
  Raw Dataset: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10
  YOLO Dataset: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format
  Models: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Models
  Results: /content/drive/My Dr

In [3]:
# ==========================================================================
# CELL 3: DOWNLOAD NWPU VHR-10 DATASET
# ==========================================================================
"""
Download NWPU VHR-10 dataset from official sources (torchgeo)
Dataset Info:
- Total: 800 images
- Positive set: 650 images (contain objects)
- Negative set: 150 images (no objects)
- Classes: 10 (airplane, ship, storage tank, baseball diamond, tennis court,
           basketball court, ground track field, harbor, bridge, vehicle)
- Resolution: 0.5-2m (Google Earth), 0.08m (Vaihingen)
"""

import zipfile
import requests
from tqdm import tqdm
import shutil

# Path where dataset will be downloaded
from pathlib import Path
dataset_raw_dir = Path(dataset_raw_dir)

print("="*80)
print("DOWNLOADING NWPU VHR-10 DATASET")
print("="*80)

# Download from torchgeo (official source)
# Note: If this link doesn't work, alternative: download from
# Google Cloud, Baidu Pan, or other mirrors
dataset_url = 'https://hf.co/datasets/torchgeo/vhr10/resolve/main/NWPU%20VHR-10%20dataset.zip'
annotations_url = 'https://hf.co/datasets/torchgeo/vhr10/resolve/main/annotations.json'

# Download dataset zip file
dataset_zip_path = dataset_raw_dir / 'NWPU_VHR-10_dataset.zip'
print(f"\n1. Downloading dataset images (this may take 5-10 minutes)...")
print(f"   URL: {dataset_url}")
print(f"   Saving to: {dataset_zip_path}")
try:
    # Download with progress bar
    response = requests.get(dataset_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(dataset_zip_path, 'wb') as f:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading dataset') as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))
    print("✓ Dataset download completed!")
except Exception as e:
    print(f"✗ Error downloading dataset: {e}")
    print("\nAlternative: Download manually from:")
    print("  - https://hf.co/datasets/torchgeo/vhr10")
    print("  - Google Cloud Storage")
    print("  - Baidu Pan (search NWPU VHR-10)")

# Download annotations JSON
annotations_path = dataset_raw_dir / 'annotations.json'
print(f"\n2. Downloading annotations file...")
print(f"   URL: {annotations_url}")
print(f"   Saving to: {annotations_path}")
try:
    response = requests.get(annotations_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    with open(annotations_path, 'wb') as f:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading annotations') as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))
    print("✓ Annotations download completed!")
except Exception as e:
    print(f"✗ Error downloading annotations: {e}")

# Extract zip file
print(f"\n3. Extracting dataset (this may take 2-3 minutes)...")
print(f"   Extracting to: {dataset_raw_dir}")
try:
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_raw_dir)
    # Remove zip file to save space
    import os
    os.remove(dataset_zip_path)
    print("✓ Dataset extraction completed!")
except Exception as e:
    print(f"✗ Error extracting dataset: {e}")

# Verify downloaded files
print(f"\n4. Verifying downloaded files...")
if annotations_path.exists():
    print(f"   ✓ Annotations: {annotations_path} ({annotations_path.stat().st_size / 1e6:.2f} MB)")

# List directory contents
print(f"\n5. Dataset directory contents:")
for item in dataset_raw_dir.iterdir():
    if item.is_dir():
        file_count = len(list(item.glob('*')))
        print(f"   📁 {item.name}/ ({file_count} items)")
    else:
        print(f"   📄 {item.name} ({item.stat().st_size / 1e6:.2f} MB)")

print("\n✓ Download and extraction completed!")


DOWNLOADING NWPU VHR-10 DATASET

1. Downloading dataset images (this may take 5-10 minutes)...
   URL: https://hf.co/datasets/torchgeo/vhr10/resolve/main/NWPU%20VHR-10%20dataset.zip
   Saving to: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10/NWPU_VHR-10_dataset.zip


✓ Dataset download completed!

2. Downloading annotations file...
   URL: https://hf.co/datasets/torchgeo/vhr10/resolve/main/annotations.json
   Saving to: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10/annotations.json


✓ Annotations download completed!

3. Extracting dataset (this may take 2-3 minutes)...
   Extracting to: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10
✓ Dataset extraction completed!

4. Verifying downloaded files...
   ✓ Annotations: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10/annotations.json (1.27 MB)

5. Dataset directory contents:
   📄 annotations.json (1.27 MB)
   📁 NWPU VHR-10 dataset/ (4 items)

✓ Download and extraction completed!


In [4]:
# ==========================================================================
# CELL 4: CONVERT COCO FORMAT TO YOLO FORMAT (CORRECTED)
# ==========================================================================
"""
Convert downloaded dataset from COCO format (JSON annotations) to YOLO format
(text files with normalized bounding box coordinates).
COCO Format: {"image_id", "bbox": [x_min, y_min, width, height], "category_id"}
YOLO Format: <class_id> <x_center_norm> <y_center_norm> <width_norm> <height_norm> (normalized to 0-1)
"""

import json
import os
from pathlib import Path
from PIL import Image
import numpy as np
import shutil  # FIX 1: Missing import!

print("="*80)
print("CONVERTING COCO FORMAT TO YOLO FORMAT")
print("="*80)

# Paths
dataset_raw_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10')
dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
annotations_path = dataset_raw_dir / 'annotations.json'

# Class mapping for NWPU VHR-10
# COCO format uses category_id (1-indexed)
class_mapping = {
    0: 'background',
    1: 'airplane',
    2: 'ship',
    3: 'storage tank',
    4: 'baseball diamond',
    5: 'tennis court',
    6: 'basketball court',
    7: 'ground track field',
    8: 'harbor',
    9: 'bridge',
    10: 'vehicle'
}

# Load COCO annotations
print("\n1. Loading COCO annotations...")
with open(annotations_path, 'r') as f:
    coco_data = json.load(f)
print(f"   ✓ Loaded {len(coco_data['images'])} images")
print(f"   ✓ Loaded {len(coco_data['annotations'])} annotations")

# Create category mapping (COCO ID -> class index for YOLO)
coco_categories = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}

# Create image ID to annotations mapping
image_annotations = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    if img_id not in image_annotations:
        image_annotations[img_id] = []
    image_annotations[img_id].append(ann)

# FIX 2: Since you have "NWPU VHR-10 dataset" -> "positive image set"
# Look for this nested structure
print("\n2. Finding positive images directory...")
positive_images_dir = None

# First try: Look for nested structure
nested_path = dataset_raw_dir / 'NWPU VHR-10 dataset' / 'positive image set'
if nested_path.exists():
    positive_images_dir = nested_path
    print(f"   ✓ Found positive images at: {positive_images_dir}")
else:
    # Fallback: Search for any directory with 'positive' in name
    for item in dataset_raw_dir.rglob('*'):
        if item.is_dir() and 'positive' in item.name.lower():
            positive_images_dir = item
            print(f"   ✓ Found positive images at: {positive_images_dir}")
            break

if positive_images_dir is None:
    print("✗ Could not find 'positive image set' directory")
    print("  Available directories:")
    for item in dataset_raw_dir.rglob('*'):
        if item.is_dir():
            print(f"    - {item}")
    raise Exception("Cannot find positive image set directory!")

# Convert annotations
print("\n3. Converting annotations to YOLO format...")

# Create images and labels directories in YOLO format directory
images_dir = dataset_processed_dir / 'images'
labels_dir = dataset_processed_dir / 'labels'
images_dir.mkdir(parents=True, exist_ok=True)  # FIX 3: Added parents=True
labels_dir.mkdir(parents=True, exist_ok=True)

conversion_count = 0
error_count = 0

for coco_image in coco_data['images']:
    try:
        img_id = coco_image['id']
        img_file_name = coco_image['file_name']
        # FIX 4: Handle missing width/height in COCO JSON
        img_width = coco_image.get('width')
        img_height = coco_image.get('height')

        # Find source image
        source_img_path = positive_images_dir / img_file_name
        if not source_img_path.exists():
            # Try with just filename (no path)
            if '/' in img_file_name:
                img_file_name = img_file_name.split('/')[-1]
                source_img_path = positive_images_dir / img_file_name

        if not source_img_path.exists():
            print(f"   ✗ Image not found: {img_file_name}")
            error_count += 1
            continue

        # FIX 5: If dimensions missing, read from actual image
        if img_width is None or img_height is None:
            with Image.open(source_img_path) as img:
                img_width, img_height = img.size

        # Copy image to processed directory
        dest_img_path = images_dir / img_file_name
        shutil.copy2(source_img_path, dest_img_path)

        # Create YOLO format annotation
        yolo_annotations = []
        if img_id in image_annotations:
            for ann in image_annotations[img_id]:
                # Extract COCO bbox (x_min, y_min, width, height)
                x_min, y_min, bbox_width, bbox_height = ann['bbox']
                # Convert to center coordinates
                x_center = x_min + bbox_width / 2
                y_center = y_min + bbox_height / 2
                # Normalize to 0-1
                x_center_norm = x_center / img_width
                y_center_norm = y_center / img_height
                width_norm = bbox_width / img_width
                height_norm = bbox_height / img_height
                # Get YOLO class ID (0-indexed, excluding background)
                coco_category_id = ann['category_id']
                yolo_class_id = coco_categories[coco_category_id]
                yolo_annotations.append(f"{yolo_class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}")

        # FIX 6: Use Path.stem to get filename without extension
        label_file_name = Path(img_file_name).stem + '.txt'
        label_path = labels_dir / label_file_name
        with open(label_path, 'w') as f:
            f.write('\n'.join(yolo_annotations))

        conversion_count += 1
        if conversion_count % 100 == 0:
            print(f"   ✓ Converted {conversion_count} images...")
    except Exception as e:
        print(f"   ✗ Error processing image {coco_image.get('file_name', 'unknown')}: {e}")
        error_count += 1

print(f"\n4. Conversion Summary:")
print(f"   ✓ Successfully converted: {conversion_count} images")
print(f"   ✗ Errors: {error_count}")

# Verify conversion
print(f"\n5. Verifying conversion...")
image_files = list(images_dir.glob('*.jpg'))
label_files = list(labels_dir.glob('*.txt'))
print(f"   ✓ Images: {len(image_files)}")
print(f"   ✓ Labels: {len(label_files)}")

# Check for mismatches
if len(image_files) != len(label_files):
    print(f"   ⚠ Warning: Mismatch between images ({len(image_files)}) and labels ({len(label_files)})")

# Display sample annotation
if label_files:
    sample_label = label_files[0]  # FIX 7: Get first label file, not list
    print(f"\n6. Sample YOLO annotation (from {sample_label.name}):")
    with open(sample_label, 'r') as f:
        content = f.read()
        if content:
            # Show only first 3 lines
            lines = content.split('\n')[:3]
            for line in lines:
                print(f"   {line}")
            if len(content.split('\n')) > 3:
                print(f"   ... ({len(content.split('\n'))} total annotations)")
        else:
            print(f"   (empty - no objects)")

print("\n" + "="*80)
print("✓ DATASET SUCCESSFULLY CONVERTED TO YOLO FORMAT!")
print("="*80)


CONVERTING COCO FORMAT TO YOLO FORMAT

1. Loading COCO annotations...
   ✓ Loaded 650 images
   ✓ Loaded 3921 annotations

2. Finding positive images directory...
   ✓ Found positive images at: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10/NWPU VHR-10 dataset/positive image set

3. Converting annotations to YOLO format...
   ✓ Converted 100 images...
   ✓ Converted 200 images...
   ✓ Converted 300 images...
   ✓ Converted 400 images...
   ✓ Converted 500 images...
   ✓ Converted 600 images...

4. Conversion Summary:
   ✓ Successfully converted: 650 images
   ✗ Errors: 0

5. Verifying conversion...
   ✓ Images: 650
   ✓ Labels: 650

6. Sample YOLO annotation (from 001.txt):
   0 0.622129 0.653465 0.068894 0.106436

✓ DATASET SUCCESSFULLY CONVERTED TO YOLO FORMAT!


In [5]:
# ==========================================================================
# CELL 5: CREATE TRAIN/VAL SPLIT AND GENERATE dataset.yaml (FIXED)
# ==========================================================================
"""
Split the dataset into training (80%) and validation (20%) sets,
and create the dataset.yaml configuration file required by YOLOv8.
CRITICAL: YOLOv8 expects this structure:
  dataset/
    images/
      train/
      val/
    labels/
      train/
      val/
"""

import os
import shutil
from pathlib import Path
import random
import yaml

print("="*80)
print("CREATING TRAIN/VAL SPLIT")
print("="*80)

# Paths
dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
images_dir = dataset_processed_dir / 'images'
labels_dir = dataset_processed_dir / 'labels'

# FIX: Create correct YOLOv8 directory structure
# YOLOv8 expects: dataset/images/train and dataset/labels/train (parallel structure)
train_images_dir = dataset_processed_dir / 'images' / 'train'
train_labels_dir = dataset_processed_dir / 'labels' / 'train'
val_images_dir = dataset_processed_dir / 'images' / 'val'
val_labels_dir = dataset_processed_dir / 'labels' / 'val'
for directory in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    directory.mkdir(parents=True, exist_ok=True)

# Get all image files from the converted dataset
all_images = sorted(list(images_dir.glob('*.jpg')))
print(f"\nTotal images: {len(all_images)}")
if len(all_images) == 0:
    print("✗ ERROR: No images found in converted dataset!")
    print(f"  Looking in: {images_dir}")
    print("  Please check CELL 4 conversion completed successfully")
    raise Exception("No images found for train/val split")

# Split into train (80%) and val (20%)
random.seed(42)  # For reproducibility
random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images = all_images[split_idx:]

print(f"Training images: {len(train_images)} (80%)")
print(f"Validation images: {len(val_images)} (20%)")

# Copy train images and labels
print("\n1. Copying training images and labels...")
train_copied = 0
train_errors = 0
for img_path in train_images:
    try:
        # Copy image
        shutil.copy2(img_path, train_images_dir / img_path.name)
        # Copy corresponding label
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy2(label_path, train_labels_dir / label_path.name)
            train_copied += 1
        else:
            print(f"   ⚠ Label not found for: {img_path.name}")
    except Exception as e:
        print(f"   ✗ Error copying {img_path.name}: {e}")
        train_errors += 1

print(f"   ✓ Copied {train_copied} training image-label pairs")
if train_errors > 0:
    print(f"   ✗ Errors: {train_errors}")

# Copy val images and labels
print("\n2. Copying validation images and labels...")
val_copied = 0
val_errors = 0
for img_path in val_images:
    try:
        # Copy image
        shutil.copy2(img_path, val_images_dir / img_path.name)
        # Copy corresponding label
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            shutil.copy2(label_path, val_labels_dir / label_path.name)
            val_copied += 1
        else:
            print(f"   ⚠ Label not found for: {img_path.name}")
    except Exception as e:
        print(f"   ✗ Error copying {img_path.name}: {e}")
        val_errors += 1

print(f"   ✓ Copied {val_copied} validation image-label pairs")
if val_errors > 0:
    print(f"   ✗ Errors: {val_errors}")

# Define class names for NWPU VHR-10 (10 classes, 0-indexed)
class_names = [
    'airplane',
    'ship',
    'storage tank',
    'baseball diamond',
    'tennis court',
    'basketball court',
    'ground track field',
    'harbor',
    'bridge',
    'vehicle'
]

# FIX: Create dataset.yaml with CORRECT paths for YOLOv8
# YOLOv8 expects paths relative to the dataset root
# Structure: dataset_root/images/train and dataset_root/labels/train
dataset_yaml = {
    'path': str(dataset_processed_dir),   # Dataset root directory
    'train': 'images/train',              # Relative path to training images
    'val': 'images/val',                  # Relative path to validation images
    'nc': len(class_names),               # Number of classes
    'names': class_names                  # Class names as list (0-indexed)
}

yaml_path = dataset_processed_dir / 'dataset.yaml'
print("\n3. Creating dataset.yaml...")
with open(yaml_path, 'w') as f:
    yaml.dump(dataset_yaml, f, default_flow_style=False, sort_keys=False)
print(f"   ✓ Created: {yaml_path}")

# Display dataset.yaml content
print("\n4. dataset.yaml content:")
print("   " + "="*70)
with open(yaml_path, 'r') as f:
    content = f.read()
    for line in content.split('\n'):
        print(f"   {line}")
print("   " + "="*70)

# Verify split and structure
print(f"\n5. Verifying dataset structure...")
train_img_count = len(list(train_images_dir.glob('*.jpg')))
train_lbl_count = len(list(train_labels_dir.glob('*.txt')))
val_img_count = len(list(val_images_dir.glob('*.jpg')))
val_lbl_count = len(list(val_labels_dir.glob('*.txt')))
print(f"   Train: {train_img_count} images, {train_lbl_count} labels")
print(f"   Val: {val_img_count} images, {val_lbl_count} labels")

# Verify parallel structure
print(f"\n6. Verifying YOLOv8 expected structure:")
expected_structure = f"""
{dataset_processed_dir.name}/
├── images/
│   ├── train/ ({train_img_count} images)
│   └── val/ ({val_img_count} images)
├── labels/
│   ├── train/ ({train_lbl_count} labels)
│   └── val/ ({val_lbl_count} labels)
└── dataset.yaml
"""
print(expected_structure)

# Check for mismatches
if train_img_count != train_lbl_count:
    print(f"   ⚠ WARNING: Train mismatch! {train_img_count} images but {train_lbl_count} labels")
if val_img_count != val_lbl_count:
    print(f"   ⚠ WARNING: Val mismatch! {val_img_count} images but {val_lbl_count} labels")
if train_img_count == train_lbl_count and val_img_count == val_lbl_count:
    print("   ✓ Perfect match! All images have corresponding labels")

# Verify at least one label has content
print(f"\n7. Verifying label content...")
sample_label_list = list(train_labels_dir.glob('*.txt'))
if sample_label_list:
    sample_label = sample_label_list[0]
    with open(sample_label, 'r') as f:
        content = f.read()
        if content.strip():
            print(f"   ✓ Sample label ({sample_label.name}) has content:")
            lines = content.strip().split('\n')[:3]
            for line in lines:
                print(f"     {line}")
            if len(content.strip().split('\n')) > 3:
                print(f"     ... ({len(content.strip().split('\n'))} total annotations)")
        else:
            print(f"   ⚠ Sample label is empty!")
else:
    print("   ⚠ No label files found in training labels directory!")

print("\n" + "="*80)
print("✓ TRAIN/VAL SPLIT COMPLETED!")
print("="*80)
print(f"\nDataset ready for YOLOv8 training!")
print(f"Next: Run CELL 6 to load model, then CELL 7 to train")


CREATING TRAIN/VAL SPLIT

Total images: 650
Training images: 520 (80%)
Validation images: 130 (20%)

1. Copying training images and labels...
   ✓ Copied 520 training image-label pairs

2. Copying validation images and labels...
   ✓ Copied 130 validation image-label pairs

3. Creating dataset.yaml...
   ✓ Created: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format/dataset.yaml

4. dataset.yaml content:
   path: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format
   train: images/train
   val: images/val
   nc: 10
   names:
   - airplane
   - ship
   - storage tank
   - baseball diamond
   - tennis court
   - basketball court
   - ground track field
   - harbor
   - bridge
   - vehicle
   

5. Verifying dataset structure...
   Train: 520 images, 520 labels
   Val: 130 images, 130 labels

6. Verifying YOLOv8 expected structure:

NWPU_VHR-10_YOLO_Format/
├── images/
│   ├── train/ (520 images)
│   └── val/ (130 images)
├── labels/
│   ├── train/ (520 labels)


In [6]:
# ==========================================================================
# CELL 5B: COMPUTE CLASS FREQUENCIES AND IDENTIFY MINORITY CLASSES
# ==========================================================================
from pathlib import Path
import json
from collections import Counter
import numpy as np

dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
train_labels_dir = dataset_processed_dir / 'labels' / 'train'
class_names = [
    'airplane','ship','storage tank','baseball diamond','tennis court',
    'basketball court','ground track field','harbor','bridge','vehicle'
]

cls_counts = Counter()
img_has_class = [Counter() for _ in range(len(class_names))]

train_images_dir = dataset_processed_dir / 'images' / 'train'
train_images = sorted(list(train_images_dir.glob('*.jpg')))

for img_path in train_images:
    lbl_path = train_labels_dir / f"{img_path.stem}.txt"
    if not lbl_path.exists():
        continue
    seen_in_image = set()
    with open(lbl_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            c = int(parts[0])
            cls_counts[c] += 1
            seen_in_image.add(c)
    for c in seen_in_image:
        img_has_class[c][img_path.name] += 1

total_instances = sum(cls_counts.values())
freq = np.zeros(len(class_names), dtype=float)
for c in range(len(class_names)):
    freq[c] = cls_counts[c] / max(1, total_instances)

print("Class counts:")
for c in range(len(class_names)):
    print(f"  {class_names[c]}: {cls_counts[c]} instances, freq={freq[c]:.6f}")

# Mark minority classes (e.g., below median frequency)
threshold = float(np.median(freq))
minority_classes = [c for c in range(len(class_names)) if freq[c] < threshold]
print("\nMinority classes (below median frequency):")
for c in minority_classes:
    print(f"  {c} -> {class_names[c]}")

# Save frequencies for later cells
np.save(dataset_processed_dir / 'class_freq.npy', freq)
with open(dataset_processed_dir / 'minority_classes.json', 'w') as f:
    json.dump(minority_classes, f)


Class counts:
  airplane: 639 instances, freq=0.196434
  ship: 246 instances, freq=0.075623
  storage tank: 568 instances, freq=0.174608
  baseball diamond: 322 instances, freq=0.098986
  tennis court: 418 instances, freq=0.128497
  basketball court: 133 instances, freq=0.040885
  ground track field: 124 instances, freq=0.038119
  harbor: 172 instances, freq=0.052874
  bridge: 105 instances, freq=0.032278
  vehicle: 526 instances, freq=0.161697

Minority classes (below median frequency):
  1 -> ship
  5 -> basketball court
  6 -> ground track field
  7 -> harbor
  8 -> bridge


In [7]:
# ==========================================================================
# CELL 6: LOAD YOLOV8 MODEL (SMALL/MEDIUM VERSION)
# ==========================================================================
"""
Load pre-trained YOLOv8 model (Small or Medium, not Nano).
Available options:
- yolov8n.pt (Nano) - 3.01M parameters - NOT RECOMMENDED for satellite data
- yolov8s.pt (Small) - 11.17M parameters - RECOMMENDED
- yolov8m.pt (Medium) - 25.86M parameters - BETTER but slower
- yolov8l.pt (Large) - 43.67M parameters - BEST but requires more GPU memory
"""

from ultralytics import YOLO
import torch

print("="*80)
print("LOADING YOLOV8 MODEL")
print("="*80)

# Check GPU memory
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n✓ GPU Memory Available: {gpu_memory:.2f} GB")
    # Recommend model based on GPU memory
    if gpu_memory >= 16:
        recommended_model = 'yolov8m.pt'
        print("  Recommendation: YOLOv8 Medium (good for satellite data with more capacity)")
    else:
        recommended_model = 'yolov8s.pt'
        print("  Recommendation: YOLOv8 Small (balanced for limited GPU)")
else:
    recommended_model = 'yolov8s.pt'
    print("\n⚠ GPU not available! Consider using Google Colab with GPU")

# Select model
# CHANGE THIS IF YOU WANT DIFFERENT SIZE
model_choice = 'yolov8s.pt'  # Options: yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt
print(f"\n1. Loading YOLOv8 model: {model_choice}")
print("   Downloading pre-trained weights (first time only)...")

# Load model - ultralytics will automatically download weights
model = YOLO(model_choice)
print(f"\n2. Model loaded successfully!")
print(f"   ✓ Model: {model_choice}")

# Display model information
print(f"\n3. Model Information:")
print(f"   Total parameters: {sum(p.numel() for p in model.model.parameters())}")
print(f"   Trainable parameters: {sum(p.numel() for p in model.model.parameters() if p.requires_grad)}")

# Display model summary
print(f"\n4. Model Architecture Summary:")
print(f"   {model.model}")

# Verify model is on GPU
if next(model.model.parameters()).is_cuda:
    print(f"\n✓ Model loaded on GPU")
else:
    print(f"\n⚠ Model loaded on CPU (training will be slow)")

print("\n✓ Model loaded successfully!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
LOADING YOLOV8 MODEL

✓ GPU Memory Available: 15.83 GB
  Recommendation: YOLOv8 Small (balanced for limited GPU)

1. Loading YOLOv8 model: yolov8s.pt

2. Model loaded successfully!
   ✓ Model: yolov8s.pt

3. Model Information:
   Total parameters: 11166560
   Trainable parameters: 0

4. Model Architecture Summary:
   DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1),

In [8]:
# ==========================================================================
# REBUILD train_expanded_list.txt FROM CURRENT TRAIN SPLIT
# (Run this BEFORE your materialize cell)
# ==========================================================================
from pathlib import Path
import numpy as np
import math
import json

dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
train_images_dir = dataset_processed_dir / 'images' / 'train'
train_labels_dir = dataset_processed_dir / 'labels' / 'train'

# If you previously computed class frequencies, load them; otherwise compute quickly here
class_names = [
    'airplane','ship','storage tank','baseball diamond','tennis court',
    'basketball court','ground track field','harbor','bridge','vehicle'
]
C = len(class_names)

# Compute class frequencies from labels
cls_counts = [0]*C
total_instances = 0
for img_path in sorted(train_images_dir.glob('*.jpg')):
    lbl_path = train_labels_dir / f"{img_path.stem}.txt"
    if not lbl_path.exists():
        continue
    with open(lbl_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            c = int(parts[0])
            if 0 <= c < C:
                cls_counts[c] += 1
                total_instances += 1

freq = np.array([(cnt / total_instances) if total_instances > 0 else 0.0 for cnt in cls_counts], dtype=float)
print("Class frequencies:", freq)

# Repeat-Factor parameters
t = 0.005

def repeat_factor_for_class(fc):
    if fc <= 0:
        return 1.0
    return max(1.0, math.sqrt(t / fc))

class_rf = [repeat_factor_for_class(fc) for fc in freq]

# Build expanded list
expanded = []
for img_path in sorted(train_images_dir.glob('*.jpg')):
    lbl_path = train_labels_dir / f"{img_path.stem}.txt"
    if not lbl_path.exists():
        continue
    present = set()
    with open(lbl_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            c = int(parts[0])
            if 0 <= c < C:
                present.add(c)
    rf_img = max([class_rf[c] for c in present], default=1.0)
    k = int(math.floor(rf_img))
    p = rf_img - k
    reps = k + (1 if np.random.rand() < p else 0)
    reps = max(1, reps)
    expanded.extend([img_path.name]*reps)

expanded_list_path = dataset_processed_dir / 'train_expanded_list.txt'
with open(expanded_list_path, 'w') as f:
    for name in expanded:
        f.write(name + '\n')

print(f"Saved: {expanded_list_path} | entries={len(expanded)} | unique={len(set(expanded))}")


Class frequencies: [    0.19643    0.075623     0.17461    0.098986      0.1285    0.040885    0.038119    0.052874    0.032278      0.1617]
Saved: /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format/train_expanded_list.txt | entries=520 | unique=520


In [9]:
# ==========================================================================
# CELL 6B (ALTERNATIVE): MATERIALIZE REPEATED TRAIN SPLIT
# ==========================================================================
from pathlib import Path
import shutil

dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
train_images_dir = dataset_processed_dir / 'images' / 'train'
train_labels_dir = dataset_processed_dir / 'labels' / 'train'

bal_images_dir = dataset_processed_dir / 'images' / 'train_bal'
bal_labels_dir = dataset_processed_dir / 'labels' / 'train_bal'
bal_images_dir.mkdir(parents=True, exist_ok=True)
bal_labels_dir.mkdir(parents=True, exist_ok=True)

expanded_list_path = dataset_processed_dir / 'train_expanded_list.txt'
with open(expanded_list_path, 'r') as f:
    expanded_train_names = [line.strip() for line in f if line.strip()]

# Populate balanced split by copying with unique suffixes
counter = {}
for name in expanded_train_names:
    stem = Path(name).stem
    counter[stem] = counter.get(stem, 0) + 1
    k = counter[stem]
    src_img = train_images_dir / name
    src_lbl = train_labels_dir / f"{stem}.txt"
    if not src_img.exists() or not src_lbl.exists():
        continue
    dst_img = bal_images_dir / f"{stem}__rep{k}.jpg"
    dst_lbl = bal_labels_dir / f"{stem}__rep{k}.txt"
    shutil.copy2(src_img, dst_img)
    shutil.copy2(src_lbl, dst_lbl)

print(f"Balanced train images: {len(list(bal_images_dir.glob('*.jpg')))}")
print(f"Balanced train labels: {len(list(bal_labels_dir.glob('*.txt')))}")


Balanced train images: 520
Balanced train labels: 520


In [10]:
# ==========================================================================
# CELL 7 (ALTERNATIVE): TRAIN USING MATERIALIZED BALANCED SPLIT
# ==========================================================================
from ultralytics import YOLO
from pathlib import Path
import yaml
import shutil

dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
yaml_path = dataset_processed_dir / 'dataset.yaml'
with open(yaml_path, 'r') as f:
    ds_cfg = yaml.safe_load(f)

# Create a temporary yaml that uses the balanced train split
tmp_yaml = dataset_processed_dir / 'dataset_balanced.yaml'
ds_bal = dict(ds_cfg)
ds_bal['train'] = 'images/train_bal'  # use balanced split
ds_bal['val'] = 'images/val'          # keep validation untouched
with open(tmp_yaml, 'w') as f:
    yaml.safe_dump(ds_bal, f, sort_keys=False)

models_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Models')
runs_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Training_Runs')

model = YOLO('yolov8s.pt')
results = model.train(
    data=str(tmp_yaml),
    epochs=120,
    imgsz=640,
    batch=16,
    patience=40,
    device=0,

    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    optimizer='SGD',
    warmup_epochs=5,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.3,
    degrees=45,
    translate=0.15,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    perspective=0.0001,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,

    save=True,
    save_period=10,
    val=True,
    verbose=True,
    project=str(runs_dir),
    name='NWPU_VHR10_yolov8s_balanced_materialized',
    exist_ok=False,
)

print("\nTraining finished.")
best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
if best_model_path.exists():
    dest = models_dir / 'yolov8s_nwpu_vhr_10_best_balanced.pt'
    shutil.copy2(best_model_path, dest)
    print(f"Saved best model: {dest}")


Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format/dataset_balanced.yaml, degrees=45, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=120, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=NWPU_VHR10_yolov8s_balanced_materialized, nbs=64, nms=False, opset=None, optimiz

In [11]:
# ==========================================================================
# CELL 8: EXTRACT AND SAVE TRAINING METRICS
# ==========================================================================
"""
Extract training metrics (loss, precision, recall, mAP) from the results
and save them as CSV files for analysis.
"""

import pandas as pd
import json
from pathlib import Path
import shutil

print("="*80)
print("EXTRACTING AND SAVING TRAINING METRICS")
print("="*80)

# Paths
runs_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Training_Runs')
results_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Results')

# Find the latest run
run_dirs = sorted(runs_dir.glob('**/'), key=lambda x: x.stat().st_mtime, reverse=True)
latest_run = None
for run_dir in run_dirs:
    if (run_dir / 'weights').exists():
        latest_run = run_dir
        break

if latest_run is None:
    print("✗ No completed training runs found!")
    print("  Please run CELL 7 first!")
else:
    print(f"✓ Found latest run: {latest_run.name}")

    # Look for results.csv (YOLOv8 saves training metrics here)
    results_csv_path = latest_run / 'results.csv'
    if results_csv_path.exists():
        print(f"\n1. Loading training metrics from {results_csv_path.name}...")
        # Load results CSV
        df_results = pd.read_csv(results_csv_path)
        # Clean column names (remove leading/trailing spaces)
        df_results.columns = df_results.columns.str.strip()
        print(f"   ✓ Loaded {len(df_results)} epochs of training data")
        print(f"   ✓ Columns: {list(df_results.columns)}")

        # Display summary statistics
        print(f"\n2. Training Summary:")
        print(f"   Total Epochs: {len(df_results)}")

        # Try to extract key metrics
        metric_columns = {
            'train_loss': ['train/box_loss', 'box_loss'],
            'val_loss': ['val/box_loss', 'box_loss'],
            'precision': ['metrics/precision(B)', 'precision'],
            'recall': ['metrics/recall(B)', 'recall'],
            'mAP50': ['metrics/mAP50(B)', 'mAP50'],
            'mAP50_95': ['metrics/mAP50-95(B)', 'mAP50-95'],
        }
        for metric_name, possible_columns in metric_columns.items():
            for col in possible_columns:
                if col in df_results.columns:
                    last_value = df_results[col].iloc[-1]
                    print(f"   • Final {metric_name}: {last_value:.6f}")
                    break

        # Save metrics to results directory
        print(f"\n3. Saving metrics to Results directory...")
        # Save full results
        results_save_path = results_dir / 'nwpu_vhr10_training_metrics.csv'
        df_results.to_csv(results_save_path, index=False)
        print(f"   ✓ Saved: {results_save_path}")

        # Save summary statistics
        summary_dict = {}
        summary_dict['Total Epochs'] = len(df_results)
        for metric_name, possible_columns in metric_columns.items():
            for col in possible_columns:
                if col in df_results.columns:
                    summary_dict[f'Final {metric_name}'] = df_results[col].iloc[-1]
                    summary_dict[f'Best {metric_name}'] = df_results[col].max() if 'loss' not in metric_name else df_results[col].min()
                    summary_dict[f'Epoch of Best {metric_name}'] = int(df_results[col].idxmax()) if 'loss' not in metric_name else int(df_results[col].idxmin())
                    break
        summary_df = pd.DataFrame([summary_dict])
        summary_path = results_dir / 'nwpu_vhr10_training_summary.csv'
        summary_df.to_csv(summary_path, index=False)
        print(f"   ✓ Saved: {summary_path}")

        # Also save the validation results
        val_results_path = latest_run / 'weights' / 'best.pt'
        if val_results_path.exists():
            print(f"\n4. Model Information:")
            print(f"   ✓ Best model: {val_results_path}")
            print(f"     Size: {val_results_path.stat().st_size / 1e6:.2f} MB")
    else:
        print(f"✗ results.csv not found in {latest_run}")
        print(f"  Available files: {list(latest_run.glob('*.*'))}")

print("\n✓ Metrics extraction completed!")


EXTRACTING AND SAVING TRAINING METRICS
✓ Found latest run: NWPU_VHR10_yolov8s_balanced_materialized

1. Loading training metrics from results.csv...
   ✓ Loaded 115 epochs of training data
   ✓ Columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']

2. Training Summary:
   Total Epochs: 115
   • Final train_loss: 0.965690
   • Final val_loss: 1.060210
   • Final precision: 0.834240
   • Final recall: 0.932340
   • Final mAP50: 0.915510
   • Final mAP50_95: 0.663830

3. Saving metrics to Results directory...
   ✓ Saved: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Results/nwpu_vhr10_training_metrics.csv
   ✓ Saved: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Results/nwpu_vhr10_training_summary.csv

4. Model Information:
   ✓ Best model: /content/drive/My Drive/YOLOv8_NWPU_VHR10/Training_Runs/NWPU_VHR10_

In [12]:
# ==========================================================================
# CELL 9: VALIDATE MODEL ON VALIDATION SET & CALCULATE PER-CLASS METRICS
# ==========================================================================
"""
Validate the trained model on the validation set and extract per-class
metrics for analysis and comparison.
"""

from ultralytics import YOLO
import pandas as pd
from pathlib import Path
import json

print("="*80)
print("VALIDATING MODEL ON VALIDATION SET")
print("="*80)

# Paths
models_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Models')
dataset_processed_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format')
results_dir = Path('/content/drive/My Drive/YOLOv8_NWPU_VHR10/Results')
yaml_path = dataset_processed_dir / 'dataset.yaml'

# Best model path
best_model_path = models_dir / 'yolov8s_nwpu_vhr_10_best_balanced.pt'
if not best_model_path.exists():
    print(f"✗ Best model not found at {best_model_path}")
    print("  Please run CELL 7 first!")
else:
    print(f"✓ Loading best model: {best_model_path.name}")

    # Load trained model
    model = YOLO(str(best_model_path))

    print("\n1. Running validation...")
    # Validate model
    val_results = model.val(
        data=str(yaml_path),
        split='val',
        imgsz=640,
        batch=16,
        verbose=True,
        save_json=True,  # Save results as JSON
    )

    print("\n2. Validation completed!")

    # Extract overall metrics
    print(f"\n3. Overall Metrics:")
    print(f"   mAP@50: {val_results.box.map50:.4f}")
    print(f"   mAP@50:95: {val_results.box.map:.4f}")
    print(f"   Precision: {val_results.box.mp:.4f}")
    print(f"   Recall: {val_results.box.mr:.4f}")

    # Create overall metrics DataFrame
    overall_metrics = {
        'Model': 'YOLOv8 Small (NWPU VHR-10)',
        'mAP@50': val_results.box.map50,
        'mAP@50:95': val_results.box.map,
        'Precision': val_results.box.mp,
        'Recall': val_results.box.mr,
        'F1_Score': 2 * (val_results.box.mp * val_results.box.mr) / (val_results.box.mp + val_results.box.mr + 1e-9),
    }
    overall_df = pd.DataFrame([overall_metrics])
    overall_metrics_path = results_dir / 'nwpu_vhr10_overall_metrics.csv'
    overall_df.to_csv(overall_metrics_path, index=False)
    print(f"\n   ✓ Saved: {overall_metrics_path}")

    # Extract per-class metrics (if available)
    print(f"\n4. Per-Class Metrics:")
    class_names = [
        'airplane', 'ship', 'storage tank', 'baseball diamond', 'tennis court',
        'basketball court', 'ground track field', 'harbor', 'bridge', 'vehicle'
    ]

    # YOLOv8 provides class-wise statistics
    if hasattr(val_results.box, 'maps'):
        per_class_data = []
        for class_id, class_name in enumerate(class_names):
            if class_id < len(val_results.box.maps):
                map_value = val_results.box.maps[class_id]
                per_class_data.append({
                    'Class': class_name,
                    'Class_ID': class_id,
                    'AP@50': map_value,
                })
                print(f"   {class_name:20s}: AP@50 = {map_value:.4f}")
        if per_class_data:
            per_class_df = pd.DataFrame(per_class_data)
            per_class_metrics_path = results_dir / 'nwpu_vhr10_per_class_metrics.csv'
            per_class_df.to_csv(per_class_metrics_path, index=False)
            print(f"\n   ✓ Saved: {per_class_metrics_path}")

print("\n✓ Validation completed successfully!")


VALIDATING MODEL ON VALIDATION SET
✓ Loading best model: yolov8s_nwpu_vhr_10_best_balanced.pt

1. Running validation...
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 64.6±32.7 MB/s, size: 94.8 KB)
val: Scanning /content/drive/My Drive/YOLOv8_NWPU_VHR10/NWPU_VHR-10_YOLO_Format/labels/val.cache... 130 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 130/130 74.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 9/9 2.2it/s 4.1s
                   all        130        668      0.893      0.927      0.936      0.681
              airplane         16        118      0.978      0.992      0.988      0.711
                  ship          8         52      0.889      0.923       0.95      0.651
          storage tank          3         94      0.934        